In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 54 kB 1.6 MB/s eta 0:00:011
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-py3-none-any.whl size=73198 sha256=1117a01e4f9c5bd5f4b6f1a27e28843848371d124a8d99c87f21b9ff309328af
  Stored in directory: /root/.cache/pip/wheels/33/f0/c8/218919df808b66696bdb2fbc2261ae74412383483226b0b31d
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=f8fd139ea81aa4eb4202c9d3a6ef8171ad48d1b593b429b500e741b05b53e560
  Stored in directory: /root/.cache/pip/wheels/86/1b/58/c23af2fe683acd8edc15d5a1268f0242be1ff2cf827fe34737
Successfully built keras-tuner terminaltables


In [3]:
import numpy as np

In [4]:
import tensorflow as tf
from tensorflow import keras

In [5]:
print(tf.__version__)

2.1.0


In [6]:
fm=keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_labels)=fm.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [8]:
train_images

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [9]:
train_images=train_images/255.0

In [10]:
test_images=test_images/255.0

In [11]:
train_images[0].shape

(28, 28)

# We have to reshape it to (28,28,1) for the input in CNN

In [12]:
len(train_images)

60000

In [13]:
train_images=train_images.reshape(len(train_images),28,28,1)

In [14]:
test_images=test_images.reshape(len(test_images),28,28,1)

In [15]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [16]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [17]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

In [18]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - ETA: 1:51:19 - loss: 2.3063 - accuracy: 0.156 - ETA: 10:11 - loss: 5.7833 - accuracy: 0.0795  - ETA: 5:07 - loss: 4.0429 - accuracy: 0.100 - ETA: 3:26 - loss: 3.4636 - accuracy: 0.10 - ETA: 2:32 - loss: 3.1547 - accuracy: 0.10 - ETA: 2:00 - loss: 2.9764 - accuracy: 0.10 - ETA: 1:41 - loss: 2.8679 - accuracy: 0.09 - ETA: 1:26 - loss: 2.7832 - accuracy: 0.09 - ETA: 1:15 - loss: 2.7209 - accuracy: 0.09 - ETA: 1:07 - loss: 2.6727 - accuracy: 0.09 - ETA: 1:01 - loss: 2.6374 - accuracy: 0.09 - ETA: 55s - loss: 2.6060 - accuracy: 0.0960 - ETA: 51s - loss: 2.5797 - accuracy: 0.096 - ETA: 47s - loss: 2.5594 - accuracy: 0.099 - ETA: 44s - loss: 2.5404 - accuracy: 0.098 - ETA: 41s - loss: 2.5254 - accuracy: 0.100 - ETA: 38s - loss: 2.5107 - accuracy: 0.103 - ETA: 36s - loss: 2.4990 - accuracy: 0.103 - ETA: 34s - loss: 2.4879 - accuracy: 0.102 - ETA: 33s - loss: 2.4778 - accuracy: 0.103 - ETA:

54000/54000 [==============================] - ETA: 9s - loss: 2.3198 - accuracy: 0.12 - ETA: 8s - loss: 2.3065 - accuracy: 0.09 - ETA: 8s - loss: 2.3063 - accuracy: 0.11 - ETA: 7s - loss: 2.3057 - accuracy: 0.10 - ETA: 7s - loss: 2.3058 - accuracy: 0.10 - ETA: 7s - loss: 2.3054 - accuracy: 0.10 - ETA: 7s - loss: 2.3047 - accuracy: 0.10 - ETA: 7s - loss: 2.3039 - accuracy: 0.10 - ETA: 7s - loss: 2.3039 - accuracy: 0.10 - ETA: 7s - loss: 2.3045 - accuracy: 0.10 - ETA: 7s - loss: 2.3041 - accuracy: 0.10 - ETA: 6s - loss: 2.3040 - accuracy: 0.10 - ETA: 6s - loss: 2.3041 - accuracy: 0.10 - ETA: 6s - loss: 2.3039 - accuracy: 0.10 - ETA: 6s - loss: 2.3038 - accuracy: 0.10 - ETA: 7s - loss: 2.3038 - accuracy: 0.10 - ETA: 7s - loss: 2.3037 - accuracy: 0.10 - ETA: 7s - loss: 2.3039 - accuracy: 0.10 - ETA: 7s - loss: 2.3041 - accuracy: 0.10 - ETA: 7s - loss: 2.3042 - accuracy: 0.10 - ETA: 7s - loss: 2.3042 - accuracy: 0.10 - ETA: 7s - loss: 2.3042 - accuracy: 0.10 - ETA: 8s - loss: 2.3041 - accu

54000/54000 [==============================] - ETA: 11s - loss: 2.3047 - accuracy: 0.062 - ETA: 7s - loss: 2.3012 - accuracy: 0.076 - ETA: 8s - loss: 2.3035 - accuracy: 0.06 - ETA: 7s - loss: 2.3038 - accuracy: 0.07 - ETA: 7s - loss: 2.3050 - accuracy: 0.08 - ETA: 7s - loss: 2.3049 - accuracy: 0.08 - ETA: 7s - loss: 2.3037 - accuracy: 0.08 - ETA: 7s - loss: 2.3036 - accuracy: 0.08 - ETA: 7s - loss: 2.3037 - accuracy: 0.09 - ETA: 7s - loss: 2.3036 - accuracy: 0.08 - ETA: 7s - loss: 2.3040 - accuracy: 0.09 - ETA: 7s - loss: 2.3038 - accuracy: 0.09 - ETA: 7s - loss: 2.3038 - accuracy: 0.09 - ETA: 6s - loss: 2.3035 - accuracy: 0.09 - ETA: 6s - loss: 2.3035 - accuracy: 0.09 - ETA: 6s - loss: 2.3037 - accuracy: 0.09 - ETA: 6s - loss: 2.3039 - accuracy: 0.09 - ETA: 6s - loss: 2.3040 - accuracy: 0.09 - ETA: 6s - loss: 2.3039 - accuracy: 0.09 - ETA: 6s - loss: 2.3037 - accuracy: 0.09 - ETA: 6s - loss: 2.3037 - accuracy: 0.09 - ETA: 6s - loss: 2.3036 - accuracy: 0.09 - ETA: 6s - loss: 2.3039 - a

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - ETA: 10:49 - loss: 2.2936 - accuracy: 0.250 - ETA: 1:01 - loss: 1.6913 - accuracy: 0.427 - ETA: 36s - loss: 1.3769 - accuracy: 0.5270 - ETA: 27s - loss: 1.1789 - accuracy: 0.586 - ETA: 23s - loss: 1.0611 - accuracy: 0.630 - ETA: 20s - loss: 0.9866 - accuracy: 0.653 - ETA: 17s - loss: 0.9300 - accuracy: 0.667 - ETA: 16s - loss: 0.8842 - accuracy: 0.685 - ETA: 15s - loss: 0.8510 - accuracy: 0.691 - ETA: 14s - loss: 0.8345 - accuracy: 0.697 - ETA: 14s - loss: 0.8094 - accuracy: 0.707 - ETA: 13s - loss: 0.7860 - accuracy: 0.714 - ETA: 13s - loss: 0.7637 - accuracy: 0.722 - ETA: 12s - loss: 0.7508 - accuracy: 0.728 - ETA: 12s - loss: 0.7337 - accuracy: 0.734 - ETA: 11s - loss: 0.7228 - accuracy: 0.739 - ETA: 12s - loss: 0.7182 - accuracy: 0.741 - ETA: 11s - loss: 0.7091 - accuracy: 0.743 - ETA: 11s - loss: 0.6987 - accuracy: 0.747 - ETA: 11s - loss: 0.6869 - accuracy: 0.751 - ETA: 10s -

54000/54000 [==============================] - ETA: 13s - loss: 0.1784 - accuracy: 0.937 - ETA: 8s - loss: 0.2682 - accuracy: 0.895 - ETA: 8s - loss: 0.2601 - accuracy: 0.90 - ETA: 8s - loss: 0.2438 - accuracy: 0.91 - ETA: 7s - loss: 0.2579 - accuracy: 0.90 - ETA: 7s - loss: 0.2593 - accuracy: 0.90 - ETA: 7s - loss: 0.2641 - accuracy: 0.90 - ETA: 7s - loss: 0.2606 - accuracy: 0.90 - ETA: 7s - loss: 0.2599 - accuracy: 0.90 - ETA: 7s - loss: 0.2627 - accuracy: 0.90 - ETA: 7s - loss: 0.2597 - accuracy: 0.90 - ETA: 7s - loss: 0.2538 - accuracy: 0.90 - ETA: 7s - loss: 0.2610 - accuracy: 0.90 - ETA: 7s - loss: 0.2596 - accuracy: 0.90 - ETA: 7s - loss: 0.2558 - accuracy: 0.90 - ETA: 7s - loss: 0.2541 - accuracy: 0.90 - ETA: 7s - loss: 0.2534 - accuracy: 0.90 - ETA: 7s - loss: 0.2537 - accuracy: 0.90 - ETA: 7s - loss: 0.2538 - accuracy: 0.90 - ETA: 7s - loss: 0.2551 - accuracy: 0.90 - ETA: 7s - loss: 0.2543 - accuracy: 0.90 - ETA: 7s - loss: 0.2566 - accuracy: 0.90 - ETA: 7s - loss: 0.2579 - a

...

54000/54000 [==============================] - ETA: 9s - loss: 0.2355 - accuracy: 0.87 - ETA: 7s - loss: 0.2332 - accuracy: 0.89 - ETA: 7s - loss: 0.3046 - accuracy: 0.87 - ETA: 7s - loss: 0.2956 - accuracy: 0.88 - ETA: 7s - loss: 0.3030 - accuracy: 0.87 - ETA: 7s - loss: 0.3064 - accuracy: 0.87 - ETA: 7s - loss: 0.2977 - accuracy: 0.88 - ETA: 7s - loss: 0.3027 - accuracy: 0.88 - ETA: 7s - loss: 0.3037 - accuracy: 0.88 - ETA: 7s - loss: 0.3074 - accuracy: 0.88 - ETA: 7s - loss: 0.3029 - accuracy: 0.88 - ETA: 7s - loss: 0.2999 - accuracy: 0.88 - ETA: 7s - loss: 0.3013 - accuracy: 0.88 - ETA: 7s - loss: 0.3032 - accuracy: 0.88 - ETA: 6s - loss: 0.3021 - accuracy: 0.88 - ETA: 6s - loss: 0.2996 - accuracy: 0.88 - ETA: 6s - loss: 0.2967 - accuracy: 0.88 - ETA: 6s - loss: 0.2940 - accuracy: 0.88 - ETA: 6s - loss: 0.2942 - accuracy: 0.88 - ETA: 6s - loss: 0.2962 - accuracy: 0.88 - ETA: 6s - loss: 0.2945 - accuracy: 0.88 - ETA: 6s - loss: 0.2932 - accuracy: 0.89 - ETA: 6s - loss: 0.2931 - accu

54000/54000 [==============================] - ETA: 13s - loss: 0.1518 - accuracy: 0.968 - ETA: 8s - loss: 0.1902 - accuracy: 0.932 - ETA: 7s - loss: 0.1910 - accuracy: 0.92 - ETA: 7s - loss: 0.2046 - accuracy: 0.91 - ETA: 7s - loss: 0.2124 - accuracy: 0.91 - ETA: 7s - loss: 0.2314 - accuracy: 0.91 - ETA: 7s - loss: 0.2228 - accuracy: 0.91 - ETA: 7s - loss: 0.2146 - accuracy: 0.91 - ETA: 7s - loss: 0.2170 - accuracy: 0.91 - ETA: 7s - loss: 0.2143 - accuracy: 0.91 - ETA: 7s - loss: 0.2162 - accuracy: 0.91 - ETA: 7s - loss: 0.2111 - accuracy: 0.91 - ETA: 7s - loss: 0.2087 - accuracy: 0.92 - ETA: 7s - loss: 0.2098 - accuracy: 0.92 - ETA: 7s - loss: 0.2091 - accuracy: 0.92 - ETA: 7s - loss: 0.2115 - accuracy: 0.92 - ETA: 7s - loss: 0.2108 - accuracy: 0.92 - ETA: 7s - loss: 0.2099 - accuracy: 0.92 - ETA: 6s - loss: 0.2114 - accuracy: 0.92 - ETA: 6s - loss: 0.2102 - accuracy: 0.91 - ETA: 7s - loss: 0.2092 - accuracy: 0.92 - ETA: 6s - loss: 0.2113 - accuracy: 0.91 - ETA: 6s - loss: 0.2145 - a

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - ETA: 10:41 - loss: 2.3042 - accuracy: 0.093 - ETA: 55s - loss: 1.4591 - accuracy: 0.4663  - ETA: 32s - loss: 1.1860 - accuracy: 0.575 - ETA: 24s - loss: 1.0258 - accuracy: 0.631 - ETA: 20s - loss: 0.9277 - accuracy: 0.666 - ETA: 17s - loss: 0.8506 - accuracy: 0.690 - ETA: 15s - loss: 0.7985 - accuracy: 0.710 - ETA: 14s - loss: 0.7688 - accuracy: 0.722 - ETA: 13s - loss: 0.7418 - accuracy: 0.732 - ETA: 12s - loss: 0.7203 - accuracy: 0.739 - ETA: 11s - loss: 0.6971 - accuracy: 0.747 - ETA: 11s - loss: 0.6787 - accuracy: 0.755 - ETA: 10s - loss: 0.6601 - accuracy: 0.763 - ETA: 10s - loss: 0.6539 - accuracy: 0.768 - ETA: 10s - loss: 0.6407 - accuracy: 0.773 - ETA: 9s - loss: 0.6330 - accuracy: 0.777 - ETA: 9s - loss: 0.6202 - accuracy: 0.78 - ETA: 9s - loss: 0.6163 - accuracy: 0.78 - ETA: 9s - loss: 0.6084 - accuracy: 0.78 - ETA: 8s - loss: 0.5991 - accuracy: 0.78 - ETA: 8s - loss: 0.5

54000/54000 [==============================] - ETA: 10s - loss: 0.1667 - accuracy: 0.937 - ETA: 7s - loss: 0.2579 - accuracy: 0.903 - ETA: 7s - loss: 0.2569 - accuracy: 0.91 - ETA: 7s - loss: 0.2580 - accuracy: 0.90 - ETA: 7s - loss: 0.2627 - accuracy: 0.90 - ETA: 7s - loss: 0.2489 - accuracy: 0.91 - ETA: 6s - loss: 0.2408 - accuracy: 0.91 - ETA: 6s - loss: 0.2398 - accuracy: 0.91 - ETA: 6s - loss: 0.2374 - accuracy: 0.91 - ETA: 6s - loss: 0.2363 - accuracy: 0.91 - ETA: 6s - loss: 0.2297 - accuracy: 0.91 - ETA: 6s - loss: 0.2275 - accuracy: 0.91 - ETA: 6s - loss: 0.2253 - accuracy: 0.91 - ETA: 6s - loss: 0.2317 - accuracy: 0.91 - ETA: 6s - loss: 0.2307 - accuracy: 0.91 - ETA: 6s - loss: 0.2304 - accuracy: 0.91 - ETA: 6s - loss: 0.2342 - accuracy: 0.91 - ETA: 6s - loss: 0.2328 - accuracy: 0.91 - ETA: 6s - loss: 0.2313 - accuracy: 0.91 - ETA: 6s - loss: 0.2329 - accuracy: 0.91 - ETA: 6s - loss: 0.2322 - accuracy: 0.91 - ETA: 6s - loss: 0.2333 - accuracy: 0.91 - ETA: 6s - loss: 0.2337 - a

54000/54000 [==============================] - ETA: 11s - loss: 0.2682 - accuracy: 0.875 - ETA: 7s - loss: 0.2140 - accuracy: 0.924 - ETA: 7s - loss: 0.1895 - accuracy: 0.93 - ETA: 7s - loss: 0.1790 - accuracy: 0.94 - ETA: 7s - loss: 0.1729 - accuracy: 0.94 - ETA: 7s - loss: 0.1744 - accuracy: 0.93 - ETA: 7s - loss: 0.1734 - accuracy: 0.93 - ETA: 7s - loss: 0.1681 - accuracy: 0.94 - ETA: 7s - loss: 0.1698 - accuracy: 0.94 - ETA: 6s - loss: 0.1632 - accuracy: 0.94 - ETA: 6s - loss: 0.1603 - accuracy: 0.94 - ETA: 6s - loss: 0.1605 - accuracy: 0.94 - ETA: 6s - loss: 0.1657 - accuracy: 0.94 - ETA: 6s - loss: 0.1662 - accuracy: 0.94 - ETA: 6s - loss: 0.1627 - accuracy: 0.94 - ETA: 6s - loss: 0.1629 - accuracy: 0.94 - ETA: 6s - loss: 0.1613 - accuracy: 0.94 - ETA: 6s - loss: 0.1599 - accuracy: 0.94 - ETA: 6s - loss: 0.1577 - accuracy: 0.94 - ETA: 6s - loss: 0.1573 - accuracy: 0.94 - ETA: 6s - loss: 0.1552 - accuracy: 0.94 - ETA: 6s - loss: 0.1535 - accuracy: 0.94 - ETA: 6s - loss: 0.1541 - a

In [19]:
model=tuner_search.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        34608     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2654304   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,690,682
Trainable params: 2,690,682
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)